    Ontology: Contains all objects
    Subclass: Contains only clases in subclass relations
    exp: Explanations of subclasses in ontologies
    (For each subclass there is a exp)

        For each chosen_explanation in exp:
            List_of_candidate_objects: Objects that are IN chosen_explanation AND that are NOT IN subclass relation
            chosen_candidate = func(List_of_candidate_objects)
            signature=chosen_candidate
            chosen_explanation_ontology_1 = forget(chosen_explanation,signature)
            generate_subclasses(chosen_explanation_ontology_1)
            #Possible for one iteration for each explanation

In [1]:
import os
from IPython.display import clear_output

In [2]:
from subprocess import Popen, PIPE, CalledProcessError
from shutil import copyfile
from os import listdir
from os.path import isfile, join
import re

In [3]:
def move_file(src,dest):
    copyfile(src,dest)
    os.remove(src)

def get_path( path):
    path = os.path.join(*path)
    return(path)

def get_all_files(mypath, _filter):
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    return ([f for f in onlyfiles if 'exp-' in f])

def get_name(name, prefix='sc', suffix = '.nt', iteration = 0):
    names= name.split('-')
    if len(names) == 1:
        return(f'{prefix}-{name}-{iteration}{suffix}')
    else:
        i = int(names[-1])
        return(f'{prefix}-{names[1]}-{i}{suffix}')
    
def get_subclasses(path, input_file):
    input_ontology = get_path([path,input_file])
    subclass_file = 'subClasses.nt'
    subclass_file_path = get_path([path,subclass_file])
    cmd = 'java -jar kr_functions.jar ' + 'saveAllSubClasses' + " " + input_ontology
    with Popen(cmd, stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout:
            print(line, end='') # process line here

    if p.returncode != 0:
        print('Error for subclass command')
    elif p.returncode == 0:
        subclass_file_path = get_path([path,subclass_file])
        new_subclass_file = get_name(input_file.strip('.owl'))
        new_subclass_path = get_path([path,new_subclass_file])
        move_file(subclass_file_path,new_subclass_path)
        print(f'Subclass file for {input_file} stored at {new_subclass_path}')
        return(new_subclass_path)
    
def get_explanations(path, input_file, subclass_file):
    input_ontology = get_path([path,input_file])
    input_subclass = get_path([path,subclass_file])
    cmd = 'java -jar kr_functions.jar ' + 'saveAllExplanations' + " " + input_ontology + " " + input_subclass
    exp_folder = get_name(input_file.strip('.owl'), suffix = '', prefix = 'exp')
    new_full_path = get_path([path,exp_folder])
    os.mkdir(new_full_path)
    with Popen(cmd, stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout:
            print(line, end='') # process line here

    if p.returncode != 0:
        print('Error for subclass command')
    elif p.returncode == 0:
#         exp_folder = get_name(input_file.strip('.owl'), suffix = '', prefix = 'exp')
#         new_full_path = get_path([path,exp_folder])
#         os.mkdir(new_full_path)
        exp_files = get_all_files(path,_filter = 'exp-')
        for file in exp_files:
            new_path = get_path([path, exp_folder, file])
            old_path = get_path([path,file])
            move_file(old_path,new_path)
        print(f'Explanation files for {input_file} stored at {new_full_path}')
        return(new_full_path)

In [24]:
def forget_from_exp(exp_path, exp_candidates, func = 'min'):

    candidate_choosing = func_dict[func]
    exp_file = exp_path.split('\\')[-1]
    new_file = get_name(exp_file, prefix = f'{func}_signatures', suffix='')
    new_path = get_path([exp_path.strip(exp_file)[:-1],new_file])


    new_ontology_name = get_name(exp_file, prefix=f'{func}_exp_ont', suffix='')
    new_ontology_path = get_path([exp_path.strip(exp_file)[:-1],new_ontology_name])
    

    
    new_subclass_name = get_name(exp_file, prefix=f'{func}_exp_sc', suffix='')
    new_subclass_path = get_path([exp_path.strip(exp_file)[:-1],new_subclass_name])

    os.mkdir(new_path)
    os.mkdir(new_ontology_path)
    os.mkdir(new_subclass_path)
    for exp,candidates in exp_candidates.items():
        chosen = None
#         chosen = candidate_choosing(candidates)

#         signature = create_signature_exp([chosen],new_path, exp)
#         forgotten = forget_exp(exp_path, exp,signature)
#         new_forgotten = get_path([new_ontology_path,forgotten.split('\\')[-1]])
#         move_file(forgotten,new_forgotten)
#         subclass=get_subclasses(new_ontology_path, new_forgotten.split('\\')[-1])
#         new_subclass = get_path([new_subclass_path, subclass.split('\\')[-1]])
#         move_file(subclass,new_subclass)
#         print(new_subclass,subclass)
#         print(new_forgotten)
#         print(signature)
#         break

        try:
            chosen = candidate_choosing(candidates)

            signature = create_signature_exp([chosen],new_path, exp)
            forgotten = forget_exp(exp_path, exp,signature)
            new_forgotten = get_path([new_ontology_path,forgotten.split('\\')[-1]])
            move_file(forgotten,new_forgotten)
            subclass=get_subclasses(new_ontology_path, new_forgotten.split('\\')[-1])
            new_subclass = get_path([new_subclass_path, subclass.split('\\')[-1]])
            move_file(subclass,new_subclass)
#             print(new_forgotten)
#             print(signature)
#             break
        except:
            print(exp,candidates)
    
    return(new_path, new_ontology_path)

        
def create_signature_exp(signature,path,ontology_file):
#     print(ontology_file, signature,path)
    name = get_name('-'+ontology_file.strip('.omn'), prefix = 'signature', suffix='.txt')
    
    signature_path = get_path([path,name])
    with open(signature_path, 'w') as f:
        for item in signature:
            f.write("%s\n" % item)
    return signature_path

def forget_exp(path,input_file, signature_file, method = 2, ext = '.omn'):
    method = str(method)
#     path = path.strip('\\datasets')
    input_ontology = get_path([path,input_file])
    signature = get_path([path,signature_file])
    cmd = 'java -cp lethe-standalone.jar uk.ac.man.cs.lethe.internal.application.ForgettingConsoleApplication --owlFile ' + input_ontology + ' --method ' + method  + ' --signature ' + signature
#     new_full_path = get_path([path,exp_folder])
#     os.mkdir(new_full_path)
    with Popen(cmd, stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout:
            print(line, end='') # process line here

    if p.returncode != 0:
        print('Error for subclass command')
    elif p.returncode == 0:
#         exp_folder = get_name(input_file.strip('.owl'), suffix = '', prefix = 'exp')
#         new_full_path = get_path([path,exp_folder])
#         os.mkdir(new_full_path)
        new_ontology = get_name('-'+input_file.strip(ext), iteration=1, prefix='ont', suffix='.owl')
        new_ontology_path = get_path([path,new_ontology])
        result_file = 'result.owl'

        move_file(result_file,new_ontology_path)
#         print(f'New ontology saved at {new_ontology_path}')
        return(new_ontology_path)    

In [5]:
def forget(path,input_file, signature_file, method = 2, ext = '.owl'):
    method = str(method)
#     path = path.strip('\\datasets')
    input_ontology = get_path([path,input_file])
    signature = get_path([path,signature_file])
    cmd = 'java -cp lethe-standalone.jar uk.ac.man.cs.lethe.internal.application.ForgettingConsoleApplication --owlFile ' + input_ontology + ' --method ' + method  + ' --signature ' + signature
#     new_full_path = get_path([path,exp_folder])
#     os.mkdir(new_full_path)
    with Popen(cmd, stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout:
            print(line, end='') # process line here

    if p.returncode != 0:
        print('Error for subclass command')
    elif p.returncode == 0:
#         exp_folder = get_name(input_file.strip('.owl'), suffix = '', prefix = 'exp')
#         new_full_path = get_path([path,exp_folder])
#         os.mkdir(new_full_path)
        new_ontology = get_name(input_file.strip(ext), iteration=1, prefix='d', suffix='.owl')
        new_ontology_path = get_path([path,new_ontology])
        result_file = 'result.owl'

        move_file(result_file,new_ontology_path)
        print(f'New ontology saved at {new_ontology_path}')
        return(new_ontology_path)    

In [6]:
from tqdm import tqdm

In [22]:
def get_min(_dict):
    return min(_dict, key=_dict.get)
def get_max(_dict):
    return max(_dict, key=_dict.get)
def get_random(_dict):
    import random
    return random.choice(list(_dict.keys()))
func_dict = {'min':get_min, 'max':get_max, 'rand':get_random}

In [8]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def get_classes_sc(file):
    candidates = []
    candidate_classes ={}
    non_candidates = []
    non_candidate_classes = []
    all_classes = extract_classes(file)
    return(all_classes)

def get_classes_owl(path):
    with open(path) as f:
        text = f.read()
    _ = text.split(r'Object Properties')
    result = []
    regex = r'".+#.+"'
    
    for i in tqdm(_[-1].split('>')):
        t = re.search(regex,i)
        if t:

            result.append(t.string[t.start()+1:t.end()-1])
    # result = list(re.finditer(regex,text))
    # result = [i.string[i.start():i.end()] for i in result]
    result_count = {}
    for i in tqdm(result):
        result_count[i] = result.count(i)
#     _keys = list(result_count.keys())
    # for i in remove:
    #     for k in _keys:
    #         if i in k.lower():
    #               result_count.pop(k, None)
    return(result_count)
    
    


def extract_classes(file, remove = ['subclassof']):
    with open(file) as f:
        text = f.read()
    result = []
    regex = r'<.+#.+'
    for i in tqdm(text.split('>')):
        t = re.search(regex,i)
        if t:

            result.append(t.string[t.start()+1:t.end()])
    # result = list(re.finditer(regex,text))
    # result = [i.string[i.start():i.end()] for i in result]
    result_count = {}
    for i in tqdm(result):
        result_count[i] = result.count(i)
    _keys = list(result_count.keys())
    for i in remove:
        for k in _keys:
            if i in k.lower():
                  result_count.pop(k, None)
        
    return(result_count)
    

In [9]:
def create_signature(signature,path,ontology_file):
    name = get_name(ontology_file.strip('.owl'), prefix = 'signature', suffix='.txt')
    signature_path = get_path([path,name])
    with open(signature_path, 'w') as f:
        for item in signature:
            f.write("%s\n" % item)
    return signature_path



In [10]:
def get_objects_not_in_sc(sc_path, owl_path, ontology_name):
    sc_classes = get_classes_sc(sc_path)
    objects_owl = get_classes_owl(owl_path)
    candidates = [_object for _object in  list(objects_owl.keys()) if _object not in list(sc_classes.keys())]
    candidate_count= {}
    for candidate in candidates:
        #gets count of everything that is in ontology but not in subclass relationship
        if ontology_name in candidate:
            candidate_count[candidate] = objects_owl[candidate]
    return(candidate_count)

In [11]:
def get_candidates_exp(path, not_sc_objects):
    exp_files = get_all_files(path, _filter = 'exp-')
    files = [get_path([path,i]) for i in exp_files]
    exp_candidates = {}
    for file,exp in zip(files,exp_files):
#         with open(file, 'r') as f:
#             text = f.read()
        classes = extract_classes(file)
        candidate_classes = {}
        for _class, count in classes.items():
            if _class in not_sc_objects.keys():
                candidate_classes[_class] = count
        exp_candidates[exp] = candidate_classes

    clear_output()
    return exp_candidates


In [13]:
path = "F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets"
input_ontology = "pizza_super_simple.owl"
subclass=get_subclasses(path, input_ontology)

Starting Program...
--------------------
Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\pizza_super_simple.owl
DONE! Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\pizza_super_simple.owl
Format : RDF/XML Syntax
--------


Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\subClasses.nt


DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\subClasses.nt
--------


Subclass file for pizza_super_simple.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\sc-pizza_super_simple-0.nt


In [14]:
candidate_objects = get_objects_not_in_sc(subclass, get_path([path,input_ontology]), 'pizza.owl')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 247/247 [00:00<00:00, 123892.98it/s]


In [16]:
explanations = get_explanations(path, input_ontology, subclass)

Starting Program...
--------------------
Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\pizza_super_simple.owl
DONE! Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\pizza_super_simple.owl
Format : RDF/XML Syntax
--------


Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\sc-pizza_super_simple-0.nt
DONE! Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\sc-pizza_super_simple-0.nt
Format : Turtle Syntax
--------


Saving all explanations for the subsumptions located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\sc-pizza_super_simple-0.nt



Computing explanation for: <http://www.co-ode.org/ontologies/pizza/pizza.owl#American> rdfs:subClassOf <http://www.co-ode.org/ontologies/pizza/pizza.owl#NamedPizza>
-> Explanation #1
	 Axiom 1. SubClassOf(<http://www.co-ode.org/ontologies/pizza/


Computing explanation for: <http://www.co-ode.org/ontologies/pizza/pizza.owl#Capricciosa> rdfs:subClassOf <http://www.co-ode.org/ontologies/pizza/pizza.owl#Food>
-> Explanation #9
	 Axiom 1. SubClassOf(<http://www.co-ode.org/ontologies/pizza/pizza.owl#Capricciosa> ObjectSomeValuesFrom(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#TomatoTopping>))
	 Axiom 2. ObjectPropertyDomain(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasIngredient> <http://www.co-ode.org/ontologies/pizza/pizza.owl#Food>)
	 Axiom 3. SubObjectPropertyOf(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#hasIngredient>)
-> Explanation #9
	 Axiom 1. SubClassOf(<http://www.co-ode.org/ontologies/pizza/pizza.owl#Capricciosa> ObjectSomeValuesFrom(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#AnchoviesTopping>))
	 Axiom 2. Obje

-> Explanation #15
	 Axiom 1. ObjectPropertyDomain(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#Pizza>)
	 Axiom 2. SubClassOf(<http://www.co-ode.org/ontologies/pizza/pizza.owl#American> ObjectSomeValuesFrom(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#TomatoTopping>))
-> Explanation #15
	 Axiom 1. SubClassOf(<http://www.co-ode.org/ontologies/pizza/pizza.owl#American> ObjectSomeValuesFrom(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#PeperoniSausageTopping>))
	 Axiom 2. ObjectPropertyDomain(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <http://www.co-ode.org/ontologies/pizza/pizza.owl#Pizza>)
-> Explanation #15
	 Axiom 1. SubClassOf(<http://www.co-ode.org/ontologies/pizza/pizza.owl#American> ObjectSomeValuesFrom(<http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping> <

In [15]:
explanations

'F:\\Documents\\VU\\KR\\kr_project2_explanation_by_forgetting\\datasets\\exp-pizza_super_simple-0'

In [17]:
exp_candidates = get_candidates_exp(explanations, candidate_objects)

In [18]:
exp_candidates

{'exp-1.omn': {},
 'exp-10.omn': {},
 'exp-11.omn': {'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping': 2,
  'http://www.co-ode.org/ontologies/pizza/pizza.owl#HamTopping': 1},
 'exp-12.omn': {},
 'exp-13.omn': {'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping': 2,
  'http://www.co-ode.org/ontologies/pizza/pizza.owl#GoatsCheeseTopping': 1},
 'exp-14.omn': {},
 'exp-15.omn': {'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping': 2,
  'http://www.co-ode.org/ontologies/pizza/pizza.owl#MozzarellaTopping': 1},
 'exp-16.omn': {'http://www.co-ode.org/ontologies/pizza/pizza.owl#FruitTopping': 1,
  'http://www.co-ode.org/ontologies/pizza/pizza.owl#NutTopping': 1,
  'http://www.co-ode.org/ontologies/pizza/pizza.owl#SauceTopping': 1},
 'exp-17.omn': {},
 'exp-18.omn': {'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping': 2,
  'http://www.co-ode.org/ontologies/pizza/pizza.owl#PrawnsTopping': 1,
  'http://www.co-ode.org/ontologies/pizza/pizza.owl#hasIngredi

In [19]:
forget_from_exp(explanations, exp_candidates,func='min')

exp-1.omn {}
exp-10.omn {}
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-11.omn...
Simple: 50.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 2, TBox: 2, RBox: 0, ABox: 0 Signature: 4
converting...
Using method 2 (uk.ac.man.cs.lethe.internal.dl.forgetting.SHQForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_signatures-pizza_super_simple-0\signature-exp-11.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.SHQForgetter$@1c9af63

Input Number of Axioms: 2
Input Average Axiom size: 5.0
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.co-ode.org/ontologies/pizza/pizza.owl#HamTopping


Forgetting...
Started at Tue Oct 13 15:19:10 PKT 2020

 |                                                | 0/1 (0.00% |                                                | 0/1 (0.00%) Forgetting HamTopp |================

DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_exp_ont-pizza_super_simple-0\subClasses.nt
--------


Subclass file for ont-exp-16.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_exp_ont-pizza_super_simple-0\sc-exp-16.nt
exp-17.omn {}
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-18.omn...
Simple: 50.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 2, RBox: 1, ABox: 0 Signature: 5
converting...
Using method 2 (uk.ac.man.cs.lethe.internal.dl.forgetting.SHQForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_signatures-pizza_super_simple-0\signature-exp-18.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.SHQForgetter$@17853da

Input Number of Axioms: 3
Input Average Axiom size: 4.0
Input Definers: 0
Input Number Restrictions: 0

Starting Program...
--------------------
Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_exp_ont-pizza_super_simple-0\ont-exp-3.owl
DONE! Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_exp_ont-pizza_super_simple-0\ont-exp-3.owl
Format : RDF/XML Syntax
--------


Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_exp_ont-pizza_super_simple-0\subClasses.nt


DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_exp_ont-pizza_super_simple-0\subClasses.nt
--------


Subclass file for ont-exp-3.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\min_exp_ont-pizza_super_simple-0\sc-exp-3.nt
exp-4.omn {}
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-5.omn...
Simple:

('F:\\Documents\\VU\\KR\\kr_project2_explanation_by_forgetting\\datasets\\min_signatures-pizza_super_simple-0',
 'F:\\Documents\\VU\\KR\\kr_project2_explanation_by_forgetting\\datasets\\min_exp_ont-pizza_super_simple-0')

In [20]:
forget_from_exp(explanations, exp_candidates,func='max')

exp-1.omn {}
exp-10.omn {}
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-11.omn...
Simple: 50.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 2, TBox: 2, RBox: 0, ABox: 0 Signature: 4
converting...
Using method 2 (uk.ac.man.cs.lethe.internal.dl.forgetting.SHQForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_signatures-pizza_super_simple-0\signature-exp-11.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.SHQForgetter$@1c9af63

Input Number of Axioms: 2
Input Average Axiom size: 5.0
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping


Forgetting...
Started at Tue Oct 13 15:21:41 PKT 2020

 |                                                | 0/1 (0.00% |                                                | 0/1 (0.00%) Forgetting hasTopp |================

DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_exp_ont-pizza_super_simple-0\subClasses.nt
--------


Subclass file for ont-exp-16.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_exp_ont-pizza_super_simple-0\sc-exp-16.nt
exp-17.omn {}
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-18.omn...
Simple: 50.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 2, RBox: 1, ABox: 0 Signature: 5
converting...
Using method 2 (uk.ac.man.cs.lethe.internal.dl.forgetting.SHQForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_signatures-pizza_super_simple-0\signature-exp-18.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.SHQForgetter$@17853da

Input Number of Axioms: 3
Input Average Axiom size: 4.0
Input Definers: 0
Input Number Restrictions: 0

DONE! Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_exp_ont-pizza_super_simple-0\ont-exp-3.owl
Format : RDF/XML Syntax
--------


Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_exp_ont-pizza_super_simple-0\subClasses.nt


DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_exp_ont-pizza_super_simple-0\subClasses.nt
--------


Subclass file for ont-exp-3.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\max_exp_ont-pizza_super_simple-0\sc-exp-3.nt
exp-4.omn {}
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-5.omn...
Simple: 50.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 2, TBox: 2, RBox: 0, ABox: 0 Signature: 4
converting...
Using method 2 (uk.ac.man.cs.lethe.internal.dl.forgett

('F:\\Documents\\VU\\KR\\kr_project2_explanation_by_forgetting\\datasets\\max_signatures-pizza_super_simple-0',
 'F:\\Documents\\VU\\KR\\kr_project2_explanation_by_forgetting\\datasets\\max_exp_ont-pizza_super_simple-0')

In [25]:
forget_from_exp(explanations, exp_candidates,func='rand')

exp-1.omn {}
exp-10.omn {}
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-11.omn...
Simple: 50.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 2, TBox: 2, RBox: 0, ABox: 0 Signature: 4
converting...
Using method 2 (uk.ac.man.cs.lethe.internal.dl.forgetting.SHQForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_signatures-pizza_super_simple-0\signature-exp-11.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.SHQForgetter$@1c9af63

Input Number of Axioms: 2
Input Average Axiom size: 5.0
Input Definers: 0
Input Number Restrictions: 0


 The following symbols will be forgotten:
http://www.co-ode.org/ontologies/pizza/pizza.owl#hasTopping


Forgetting...
Started at Tue Oct 13 15:24:04 PKT 2020

 |                                                | 0/1 (0.00% |                                                | 0/1 (0.00%) Forgetting hasTopp |===============

DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_exp_ont-pizza_super_simple-0\subClasses.nt
--------


Subclass file for ont-exp-16.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_exp_ont-pizza_super_simple-0\sc-exp-16.nt
exp-17.omn {}
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-18.omn...
Simple: 50.00%, EL: 100.00%, ALC: 100.00%, ALCI: 100.00%, SHQ: 100.00%, axioms: 3, TBox: 2, RBox: 1, ABox: 0 Signature: 5
converting...
Using method 2 (uk.ac.man.cs.lethe.internal.dl.forgetting.SHQForgetter$)
Using signature file F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_signatures-pizza_super_simple-0\signature-exp-18.txt
Using forgetter uk.ac.man.cs.lethe.internal.dl.forgetting.SHQForgetter$@17853da

Input Number of Axioms: 3
Input Average Axiom size: 4.0
Input Definers: 0
Input Number Restrictions

Starting Program...
--------------------
Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_exp_ont-pizza_super_simple-0\ont-exp-3.owl
DONE! Loading ontology located at: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_exp_ont-pizza_super_simple-0\ont-exp-3.owl
Format : RDF/XML Syntax
--------


Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_exp_ont-pizza_super_simple-0\subClasses.nt


DONE! Saving all subClassOf statements to following file: F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_exp_ont-pizza_super_simple-0\subClasses.nt
--------


Subclass file for ont-exp-3.owl stored at F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\rand_exp_ont-pizza_super_simple-0\sc-exp-3.nt
exp-4.omn {}
Parsing F:\Documents\VU\KR\kr_project2_explanation_by_forgetting\datasets\exp-pizza_super_simple-0\exp-5.omn...
Si

('F:\\Documents\\VU\\KR\\kr_project2_explanation_by_forgetting\\datasets\\rand_signatures-pizza_super_simple-0',
 'F:\\Documents\\VU\\KR\\kr_project2_explanation_by_forgetting\\datasets\\rand_exp_ont-pizza_super_simple-0')